In [1]:
from init import *
from music21 import converter, instrument, note, chord
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint

autotime loaded.


Using TensorFlow backend.


In [2]:
class Music():
    def __init__(self, filenames):
        self.files = filenames
        
    def get_musical_notes(self):
        notes = []
        for filename in self.files:
            song = converter.parse(filename)
            parts = instrument.partitionByInstrument(song)
            notes_to_parse = None
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = song.flat.notes
                
            for el in notes_to_parse:
                if isinstance(el, note.Note):
                    notes.append(str(el.pitch))
                elif isinstance(el, chord.Chord):
                    chords = '.'.join(str(n) for n in el.normalOrder)
                    notes.append(chords)
        return notes
    
    def get_encoded_notes(self):
        notes = self.get_musical_notes()
        self.unique_notes = sorted(set(notes))
        self.pitchdict = {note:number for number, note in enumerate(self.unique_notes)}
        return list(map(self.pitchdict.get, notes))
    
    def get_training_sequences(self, sequence_length=100):
        ins = []
        out = []
        notes = self.get_encoded_notes()
        for i in range(0, len(notes) - sequence_length, 1):
                ins.append(notes[i:i+sequence_length])
                out.append(notes[i+sequence_length])
        ins = np.expand_dims(np.array(ins), axis=2) / float(len(self.unique_notes))
        out = to_categorical(out)
        return ins, out
    
    def compile_model(self, inputs, latent_dim=256):
        shape = (inputs.shape[1], inputs.shape[2])
        model = Sequential()
        model.add(LSTM(latent_dim, input_shape=shape, recurrent_dropout=0.3, return_sequences=True))
        model.add(LSTM(2 * latent_dim, recurrent_dropout=0.3, return_sequences=True))
        model.add(LSTM(latent_dim))
        model.add(BatchNorm())
        model.add(Dropout(0.3))
        model.add(Dense(latent_dim))
        model.add(Activation('relu'))
        model.add(BatchNorm())
        model.add(Dropout(0.3))
        model.add(Dense(len(self.unique_notes)))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        self.model = model
        return model
    
    def train(self, X, y, checkpoint='best_model.h5', epochs=100, batch_size=128, split=0.01, verbose=0):
        mc = ModelCheckpoint(checkpoint, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        self.history = self.model.fit(X, y, epochs=epochs, batch_size=batch_size, 
                                      validation_split=split, verbose=verbose, 
                                      callbacks=[mc])
        return self.history

time: 9.13 ms


In [ ]:
epochs = 50
batch_size = 256
latent_dim = 256
sequence_length = 100
checkpoint = 'best_model.h5'
files = glob.glob('./songs/*.mid')[:10]
music = Music(files)
notes = music.get_encoded_notes()
ins, out = music.get_training_sequences(sequence_length)
model = music.compile_model(ins, latent_dim)
print(model.summary())
hist = music.train(ins, out, checkpoint, epochs, batch_size, verbose=1, split=0.02)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_______________________________________